In [10]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import talos
%matplotlib inline

In [12]:
# load dataset
train = ImageDataGenerator(rescale=1/255)
test = ImageDataGenerator(rescale=1/255)

train_dataset = train.flow_from_directory("forest_fire/Training and Validation/",
                                          target_size=(150,150),
                                          batch_size = 32,
                                          class_mode = 'binary')
                                         
test_dataset = test.flow_from_directory("forest_fire/Testing/",
                                          target_size=(150,150),
                                          batch_size =32,
                                          class_mode = 'binary')

Found 1832 images belonging to 2 classes.
Found 68 images belonging to 2 classes.


In [13]:
test_dataset.class_indices

{'fire': 0, 'nofire': 1}

# Training & Testing the MLP models

In [22]:
# Input data and params into the function
def MLP_model(train_dataset, test_dataset , x_val, y_val, params):
    
    from tensorflow.keras.layers import Dense, Dropout, Flatten
    from tensorflow.keras.models import Sequential

    model = Sequential()
    model.add(Dense(params['hidden_units'], input_dim=input_size,
                    activation=params['first_activation']))
    model.add(Dropout(params['dropout']))
    
    
    ## hidden layers if any
    for i in range(params['hidden_layers']):
        model.add(Dense(params['hidden_units'], 
                        activation=params['activation']))
        model.add(Dropout(params['dropout']))
        

    model.add(Dense(num_labels, activation=params['last_activation']))
    
    model.compile(loss=params['losses'],
                  optimizer=params['optimizer'],
                  metrics=['accuracy'])
    
    history = model.fit(train_dataset, 
                        batch_size=params['batch_size'],
                        epochs=params['epochs'],
                        callbacks=[talos.callbacks.ExperimentLog('MLP', params)],
                        verbose=0)

    return history, model

In [19]:
# Set the parameter space
p = {
     'hidden_units':[256, 512],
     'hidden_layers':[0, 1, 2],
     'batch_size': [128, 256],
     'epochs': [10],
     'dropout': [0, 0.5],
     'optimizer': ['Adam'],
     'losses': ['categorical_crossentropy'],
     'first_activation': ['relu'],
     'activation':['relu', 'sigmoid'],
     'last_activation': ['relu','softmax']
    }

In [24]:
# Run the experiment
t = talos.Scan(train_dataset,
               model=MLP_model,
               params=p,
               experiment_name='MLP',
               round_limit=2)

TypeError: __init__() missing 1 required positional argument: 'y'

In [1]:
talos.Analyze(t).plot_line('val_accuracy') # Plot of accuracies of models

NameError: name 'talos' is not defined

In [2]:
talos.Analyze(t).plot_line('val_loss') # Plot of losses of models

NameError: name 'talos' is not defined

In [3]:
idx = t.data['val_accuracy'].astype('float').argmax()  # Index of Maximum Accurate MLP
t.data.loc[idx] # Hyperparameters of the best MLP

NameError: name 't' is not defined

In [4]:
# Restoring the best MLP and testing it
model = model_from_json(t.saved_models[idx])
model.set_weights(t.saved_weights[idx])
model.compile(loss=t.data.loc[idx,'losses'],
              optimizer=t.data.loc[idx,'optimizer'],
              metrics= ['accuracy'])

los,acc=model.evaluate(x_test, y_test, verbose=0)
print("\nTest Accuracy: %.1f%%" % (100.0 * acc))
print("Test Loss: %.1f%%" % (100.0 * los))

NameError: name 'model_from_json' is not defined